## Homework 4 - Programming advanced quantum algorithms

## Preamble

The purpose of this homework is to deepen your understanding of quantum algorithms and further familiarize you with quantum programming. You're going to use the concepts covered so far to implement two quantum algorithms that yield an advantage over classical algorithms, in tasks 1 and 2. Then, in task 3, you will implement another sampling problem that is a candidate for quantum computational supremacy. Finally, task 4 deals with an application of quantum computing that is more speculative in the advantages that it offers over classical computation, namely quantum machine learning.

Your solutions should be implemented in Q#, however, for tasks 1 and 3B you will receive additional bonus points if you also implement the solutions in Qiskit (the quantum computing framework used by IBM for their devices). Note that these bonus points are **conditioned on having implemented the task in Q#**. To do this, follow the instructions [here](https://qiskit.org/documentation/install.html) for installing Qiskit and setting up an IBMQ account. Access to the IBM devices is free (though there is a queue whenever you submit a task). You can also use the [example code](https://github.com/agheorghiu/cs101-3/tree/master/Lecture%2010/RCS%20on%20IBM) from lecture 10 as a reference for how to submit jobs to the IBM devices.

The total number of points without the IBMQ bonus is **140pt** and with the bonus points it's **165pt**.

For each task you have to write some Q# code but also a brief explanation of why your solution works. You will be graded on both ($70\%$  of the points will be for the code and $30\%$ for the written explanation). The written explanation can be either in the form of comments in your code or as a separate description in a text file (or both).

To turn in your homework, submit a zip file with your solutions on Moodle.

The deadline is: **May 25, 23:59 PST**.

<span style="color:red"> **Important!** </span> The only restriction in terms of using Q# is that for Task 4 you cannot use the package Microsoft.Quantum.MachineLearning. Otherwise, you are free to use any of the functionalities in Q# (as well as Python, C# or Qiskit when necessary).

### (30pt + 10pt) Task 1 - Hidden linear function problem

One of the problems we've seen for which quantum computation offers an advantage over classical computation is the Bernstein-Vazirani problem. Recall that the problem was to find a hidden $s \in \{0, 1\}^n$ from queries to $f : \{0, 1\}^n \to \{0, 1\}$, promised that $f(x) = x \cdot s$. We saw that there is a quantum algorithm that solves this problem with just one query to $f$ (or rather the unitary $U_f$), whereas classical algorithms require $\Omega(n)$ queries.

Of course, the Bernstein-Vazirani problem is an oracle problem and we'd like to see examples of quantum advantage for **non-oracle problems** as well. In this task, you will implement a quantum algorithm for such a problem, inspired by Bernstein-Vazirani, known as the _Hidden Linear Function (HLF) problem_. The problem was introduced in [this paper](https://arxiv.org/abs/1704.00690) and we take its description from [here](https://cirq.readthedocs.io/en/stable/tutorials/hidden_linear_function.html).

The problem is the following. You are given as input an $n \times n$ binary matrix $A$ and a binary vector $b$ of length $n$. We write this as $A \in \mathbb{F}_2^{n \times n}$, $b \in \mathbb{F}_2^n$ where $\mathbb{F}_2$ is the field of integers mod 2. Next, we define the quadratic form, $q: \mathbb{F}_2^n \to \mathbb{Z}_4$:

$$q(x) = 2x^T A x + b^T x \; mod \; 4$$ 

In other words, $q$ can be written as:

$$q(x) = 2 \sum_{i, j} A_{i,j} x_i x_j + \sum_i b_i x_i \; mod \; 4$$

Finally, consider the set of binary strings:

$$ \mathcal{L}_q = \{ x \in \mathbb{F}_2^n | q(x \oplus y) = q(x) + q(y) \; mod \; 4, \text{ for all } y \in \mathbb{F}_2^n  \}$$

If we restrict $q$ to elements from $\mathcal{L}_q$, then $q$ can be written as a linear function:

$$ q(x) = 2 z^{T} x \; mod \; 4$$

for some $z \in \mathbb{F}_2^n$. The task is to find this $z$, given $A$ and $b$.

This problem is interesting because it shows an unconditional separation between constant depth quantum circuits and constant depth classical circuits. There is a constant depth quantum circuit (i.e. a quantum circuit whose depth is fixed and independent of $n$) that can solve this problem, but classically one requires circuits of depth $\Omega(\log(n))$. 

Implement the quantum circuit that solves this problem in Q# by completing the code block below:

In [4]:
operation hiddenLinearFunction(A: Int[][], b: Int[]) : Int[] {
    let n = Length(b);
    let z = new Int[n];

    // your code here

    return z;
}

hiddenLinearFunction

The quantum circuit that solves this problem acts on $n$ qubits (the same $n$ as for the $n \times n$ matrix $A$ and $n$-dimensional vector $b$) initialized as $|0^n \rangle$ and is simply $H^{\otimes n} U_q H^{\otimes n}$, where $U_q$ is defined as follows:

$$ U_q = \prod_{i < j} CZ_{i,j}^{A_{i,j}} S(b) $$

Here $CZ$ is the controlled-$Z$ operation and $S$ is the single-qubit phase operation that maps $|0 \rangle$ to $|0 \rangle$ and $|1 \rangle$ to $i |1 \rangle$ (note that $S$ can also be written as $\sqrt{Z}$ or $R_Z(\pi/2)$ where $R_Z$ is the rotation around $Z$ operation). The notation $CZ_{i, j}^{A_{i, j}}$ means that we apply $CZ$ between qubits $i$ and $j$ if $A_{i, j}=1$ and we don't otherwise. Similarly, the notation $S(b)$ indicates the $n$-qubit operation

$$S(b) = S^{b_0} \otimes S^{b_1} \otimes ... \otimes S^{b_{n-1}}$$

As an example, for $n=3$ and $b = (0, 1, 1)$, $S(b) = S^0 \otimes S^1 \otimes S^1 = I \otimes S \otimes S$. In other words, apply nothing on the first qubit and apply $S$ on the last two qubits.


Note that in circuit form the order of the operations is reversed! That is, you first apply $H^{\otimes n}$ followed by $S(b)$, followed by the $CZ$ operations, followed by $H^{\otimes n}$. The output is then measured in the computational basis. When you measure the output you should obtain only $z$ values for which

$$ q(x) = 2 z^{T} x \; mod \; 4$$

You can test your solution using the code from [here](https://cirq.readthedocs.io/en/stable/tutorials/hidden_linear_function.html). Use it to generate instances of HLF and see if the $z$ values obtained are correct. Test it specifically with the example:

In [3]:
operation HLFinstance(): (Int[][], Int[]) {    

    let A = [[0, 1, 1, 0, 0, 1, 0, 0, 1, 1],
              [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
              [0, 0, 0, 0, 0, 0, 1, 1, 0, 1],
              [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
              [0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
              [0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
              [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]];
    let b = [0, 0, 0, 0, 1, 1, 1, 0, 0, 1];

    return (A, b);
}

HLFinstance

For this problem instance, there are 64 correct $z$ values. Run your code multiple times until you see all 64 values.

#### (10pt) Bonus
Implement the circuit for the above instance on the IBM 16-qubit device and collect the results. Include that as part of your submission.

### (30pt) Task 2 - Discrete logarithm

In the lectures, we've seen Shor's algorithm for factoring integers. We saw that the key component was to reduce the problem to period (or order) finding and then solve order finding quantumly. In this task, you need to solve another problem that reduces to period finding, namely computing the _discrete logarithm_. This problem, much like factoring, is tremendously important in cryptography as a number of cryptographic protocols (such as Diffie-Hellman key-exchange) rely on the computational hardness of computing the discrete logarithm.

If we again consider the group of integers modulo $N$, under multiplication, the discrete logarithm of $b$ in base $a$ is the smallest integer $s$, such that 

$$a^s = b \; mod \; N.$$

The goal is thus, given $a$, $b$ and $N$, to find $s$. As with factoring, we solve this problem by reducing it to period finding.

Consider the function 

$$f(x_1, x_2) = b^{x_1} a^{x_2} \; mod \; N.$$

Note that this is a function in two arguments, in contrast to factoring where we considered the function $f(x) = g^x \; mod \; N$. Since we know that $b = a^s$, we can rewrite this as $f(x_1, x_2) = a^{sx_1 + x_2}$. Note that for integer $0 \leq l < N$, we have that

$$f(x_1 + l, x_2 - ls) = a^{sx_1 + sl + x_2 - ls} = a^{sx_1 + x_2} = f(x_1, x_2)$$

The function is thus periodic of period $(l, -ls)$. Finding the period of the function will then give us the value $s$. 

Start by implementing the unitary performing $f(x_1, x_2)$ in Q#. Do this in analogy to the unitary we considered for $g^x \; mod \; N$, since this will make things easier. Complete the code block:

In [8]:
// maps |x1>|x2>|y> to |x1>|x2>|b^x1 * a^x2 * y mod N>
operation doubleModularExp(a: Int, b: Int, N: Int, x1: Qubit[], x2: Qubit[], y: Qubit[]) : Unit {

    // your code here

}

doubleModularExp

Next, perform period finding, as we did in Shor's algorithm. Start by creating the state

$$ \sum_{x_1} \sum_{x_2} |x_1 \rangle |x_2 \rangle |f(x_1, x_2) \rangle $$

then apply the QFT to the $|x_1 \rangle |x_2 \rangle$ registers and measure those registers in the computational basis. With high probability you should obtain $(s l_1 / 2^n, l_2 / 2^n)$, for some integers $l_1$, $l_2$, and where $n$ is the number of qubits of _one_ of the $x$ registers. Do this by completing the code block:

In [9]:
// returns the tuple (s*l1/2^n, l2/2^n); here n denotes the number of qubits for each of the x registers
operation discreteLogOrderFinder(a: Int, b: Int, N: Int, n: Int) : (Int, Int) {

    // your code here

    return (0, 0);
}

discreteLogOrderFinder

Finally, take the result in your host program (whether it's Python, C# or just the Jupyter notebook itself) and use continued fraction expansion on that result to extract $s$. You are free to use the code for continued fraction expansion from [lecture 7](https://github.com/agheorghiu/cs101-3/tree/master/Lecture%207/Quantum%20Shor's%20algorithm) or use one of the continued fraction expansion functions from Q#, such as [this one](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.canon.continuedfractionconvergent), for instance.

For a more detailed explanation of how the algorithm works, see section 5.4.2 in [Nielsen and Chuang](http://mmrc.amss.cas.cn/tlb/201702/W020170224608149940643.pdf).

### (30pt + 15pt) Task 3 - Instantaneous Quantum Polynomial-time (IQP) computation

In lecture 10, we discussed the concept of quantum computational supremacy. We said that this generally involves sampling problems that can be solved efficiently quantumly but for which we have plausible complexity-theoretic evidence that there's no efficient classical algorithm. We considered the problem of Random Circuit Sampling (RCS), while in this task we're going to consider a related problem which we'll call Random IQP Sampling (RIQPS).

IQP stands for Instantaneous Quantum Polynomial-time computation and is a class of quantum circuits having the form:

$$ H^{\otimes n} D H^{\otimes n}$$

where $D$ is a unitary operation that is diagonal in the computational basis (or $Z$ basis). Examples of operations diagonal in the computational basis that we've seen so far include the $Z$ gate, the $CZ$ gate as well as single-qubit $Z$ rotations, denoted $R_Z$.

It was shown in [this paper](https://arxiv.org/abs/1005.1407) that the existence of a polynomial-time classical algorithm that can sample from the output distribution of an IQP circuit (acting on $|0^n \rangle$ and with measurements in the computational basis) implies a collapse of the polynomial hierarchy.

In this task, you're going to examine the statistical properties of random IQP circuits. These random IQP circuits will be produced in two ways.

#### (15pt) 3A: Exponentiated Pauli Zs

Implement the operation $H^{\otimes n} D H^{\otimes n}$ as a sequence of exoponentiated $Z$ operations by completing the code block below:

In [2]:
operation iqpExp(qs: Qubit[], layers: Int[][], thetas: Double[]): Unit {    

    // your code here

}

iqpExp

Assume ``qs`` consists of $n$ qubits and ``thetas`` consists of $m$ real values. Here $m$ is going to denote the depth of the circuit $D$ and ``layers`` is an $n \times m$ binary matrix. $D$ will be implementated as a product of exponentiated Pauli $Z$s as follows. Let $Z(layer[i])$ denote the operation:

$$ Z(layer[i]) = Z^{layer[i][0]} \otimes Z^{layer[i][1]} \otimes ... \otimes Z^{layer[i][n - 1]}$$

As an example, suppose $layer[0] = (0, 1, 1, 1, 0)$, for $n = 5$, then

$$ Z(layer[0]) = I \otimes Z \otimes Z \otimes Z \otimes I.$$

$D$ is then defined a follows:

$$ D = exp(-i \theta_0 Z(layer[0])) \; exp(-i \theta_1 Z(layer[1])) \; ... \; exp(-i \theta_{m-1} Z(layer[m - 1])) $$

where $\theta_i$ are the elements of ``thetas``. You can use the Q# function [Exp](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.intrinsic.exp) to perform the exponentiated $Z$ operations.

With the implemented function, write a Python host program (or do this in the Jupyter notebook or in a C# host program) to run a number (say 500) of random IQP circuits on the $|0^n \rangle$ state and measure the output in the computational basis. You can take $n$ to be $5$. For each circuit, generate the ``layers`` and ``thetas`` values uniformly at random. For ``layers`` you simply need to generate a random binary matrix. For ``thetas`` you can assume each $\theta$ value is drawn uniformly at random from the set $\{0, \pi/8, 2\pi/8, 3\pi/8, ... 7\pi/8 \}$. Run each circuit multiple (say 200) times and collect the counts for all possible $2^n$ outputs. We'll refer to this as the **approximate output distribution**. You can use the [code from lecture 10](https://github.com/agheorghiu/cs101-3/tree/master/Lecture%2010) as a template for doing this.

Finally, use the statistics collected to compute two things. First, compute the output entropies for the approximate output distributions you obtained. In other words, for each distribution compute:

$$ S = - \sum_{i = 0}^{2^n - 1} p(i) log_2(p(i)) $$

where $p(i)$ is your estimate for the output probability of $i$ (given by the number of counts for $i$ divided by the number of runs of the circuit). Plot the entropy values you obtained.

Second, create a histogram of the $p(0)$ values for all circuits, like the one we made in lecture 10 for random quantum circuits. Here $p(0)$ denotes the probability of output $|0^n \rangle$. Do these values still obey a Porter-Thomas (exponential) distribution as was the case for general random circuits?

#### (15pt) 3B: Single and two-qubit gates

Again, you need to perform $H^{\otimes n} D H^{\otimes n}$, except this time $D$ will be implemented as cycles of single and two-qubit gates as we did for RCS. Complete the code block below:

In [3]:
operation iqpGates(qs: Qubit[], singleQubitGates: Int[][], twoQubitGates: (Int, Int)[]): Unit {

  // your code here

}

iqpGates

As with RCS, ``singleQubitGates`` is a matrix of the single-qubit gates that are applied in the cycles of the circuit, while ``twoQubitGates`` is a list of indices for the two-qubit gates. The number of cycles is given by the length of either ``singleQubitGates`` or ``twoQubitGates``. Assume that $0 \leq $ ``singleQubitGates[i][j]`` $ \leq 7$. If ``singleQubitGates[i][j] = k`` you need to apply the gate $R_Z(k \; \pi/4)$ to qubit ``j`` in cycle ``i``. For the two-qubit gates we're going to use the $CZ$ operation and ``twoQubitGates[i] = (a, b)`` specifies the two indices on which the $CZ$ gate acts. In other words, in cycle $i$ you need to apply $CZ$ on qubits $a$ and $b$ of ``qs``.

As in 3A, create a host program to generate and run a number of random IQP circuits for which you need to compute the entropies of the output distributions and plot them as well as the probabilities of seeing output $|0^n \rangle$ for each circuit, for which you should make a histogram. How do the results compare to the ones from 3A?

Finally, compute the entropies and the $p(0)$ plots for the case where you **don't have** two-qubit gates in the circuits. That is, you perform only the single-qubit gates and do not perform any two-qubit gates (you can simply comment out the application of the $CZ$ gates from your code). Do the plots differ from the previous case? 

#### (15pt) Bonus

Perform task 3B on one of the IBM devices (in this case, you can use one of the 5-qubit devices, though make sure your $CZ$ gates respect the topology of the chip on which you're running).

### (50pt) Task 4 - A simple quantum classifier

In this task, you're going to implement a simplified version of the classifier described in [this paper](https://arxiv.org/abs/1804.00633). We're going to consider a _parametrized_ quantum circuit, similar to the IQP ones. The goal will be to find the optimal parameters for the circuit so that it (almost) correctly classifies data that we feed into the circuit, just like a neural network. Schematically, the circuit will look like this:

![image.png](img/task4.png)

The unitary $S_x$ prepares the input $x$ that we want to classify. That state is acted upon by $U(\theta)$ and the top qubit is measured. Repeating this many times and taking the majority vote of the top qubit gives us the classification of $x$ (for simplicity we're assuming that we only have two classes). Your goal will be to implement $S_x$ and $U(\theta)$ and then optimize over $\theta$ so as to perform classification optimally.

#### (30 pt) Task 4A: Encoding the input

In this subtask, you need to implement the unitary $S_x$. We're going to consider the input $x$ to be a real vector of dimension $2^n$, where $n$ is the number of qubits. In other words, $x \in \mathbb{R}^{2^n}$. We want to encode $x$ into a state $| \phi(x) \rangle$ whose amplitudes are the (rescaled) components of $x$. Essentially, after applying $S_x$ on $| 0^n \rangle$ we should obtain the state:

$$ |\phi(x) \rangle = S_x |0^n \rangle = \frac{1}{||x||^2} \sum_{i = 0}^{2^n - 1} x_i |i \rangle $$

where $||x||^2$ is the $L_2$ (or Euclidean) norm of $x$.

For a description of how to do this, see the bottom of page 28 from [this](https://www.scottaaronson.com/barbados-2016.pdf). The idea is to perform the mapping recursively. Start by first computing

$$ L = \sqrt{\sum_{i = 0}^{2^{n-1} - 1} x^2_i} \quad \quad R = \sqrt{\sum_{i = 2^{n-1}}^{2^{n} - 1} x^2_i} $$

Note that $||x||^2 = L^2 + R^2$. Then use the $R_Y(\alpha)$ rotation, whose matrix form is

$$R_Y(\alpha) =  \begin{bmatrix}
cos(\alpha/2) & -sin(\alpha/2) \\
sin(\alpha/2) & cos(\alpha/2) \\
\end{bmatrix} $$

to map a $|0 \rangle$ qubit to $ \frac{1}{||x||^2}(L |0 \rangle + R |1 \rangle)$.

Next, having appended a $|0 \rangle$ qubit to this state, the task is to do the same for the left component and the right component. So $ |00 \rangle$ needs to split into $ LL |00 \rangle + LR |01 \rangle$, where $LL$ and $LR$ are

$$ LL = \sqrt{\sum_{i = 0}^{2^{n-2} - 1} x^2_i} \quad \quad LR = \sqrt{\sum_{i = 2^{n-2}}^{2^{n - 1} - 1} x^2_i} $$

These are basically the analogues of $L$ and $R$ for the first half of $x$. Similarly, $|10 \rangle$ will split into $RL |10 \rangle + RR|11 \rangle$, where

$$ RL = \sqrt{\sum_{i = 2^{n-1}}^{2^{n-1} + 2^{n-2} - 1} x^2_i} \quad \quad LR = \sqrt{\sum_{i = 2^{n-1} + 2^{n-2}}^{2^{n} - 1} x^2_i} $$

And so the state after the second iteration is

$$ \frac{1}{||x||^2} ( LL |00 \rangle + LR |01 \rangle + RL |10 \rangle + RR |11 \rangle ) $$

The process continues until the state $|\phi(x) \rangle$ is obtained.

Implement the unitary performing this operation by completing this code block:

In [4]:
operation encodeInput(x: Double[], qs: Qubit[]): Unit {

  // your code here

}

encodeInput

Note here that the length of ``qs`` is $n$ (the number of qubits), whereas the length of ``x`` is $2^n$. You can test your functionality using the ``DumpMachine`` or ``DumpRegister`` functions to print the amplitudes of your quantum state, after applying ``encodeInput`` on $|0^n \rangle$. These amplitudes should be the scaled versions of the $x_i$'s.

#### (10pt) Task 4B: Implementing the paramterized unitary

For the parametrized unitary, you are free to implement it however you like, and you're encouraged to experiment with different choices. As explained in [this paper](https://arxiv.org/abs/1804.00633), the circuit has to be strongly entangling for the classifier to work. This means that there need to be sufficient two qubit gates in the circuit for correlations to spread between _all_ qubits.

One design you can consider is inspired by IQP (specifically task 3B). You can alternate between layers of parametrized single-qubit rotation gates, followed by parametrized two-qubit gates. For the parametrized two-qubit gates you can consider controlled-rotation operations, like $CR_X(\alpha)$, for instance. This performs a controlled $R_X(\alpha)$ gate on the second qubit conditioned on the value of the first qubit.

Alternatively, you can consider exponentiated tensor products of Paulis as in task 3A.

See also [this page](https://docs.microsoft.com/en-us/quantum/libraries/machine-learning/design) for some helpful suggestions.

Give a short description of the choice of parametrized unitary you opted for and complete the code block below:

In [10]:
operation parametrizedUnitary(qs: Qubit[], thetas: Double[]): Unit {

  // your code here

}

parametrizedUnitary

Note that you don't necessarily need to adhere to this template. You can add more arguments to ``parametrizedUnitary`` if you find it helpful.

#### (10pt) Task 4C: Implementing the classifier

Using the code from 4A and 4B, implement the classifier. You are given a dataset containing a number of training examples as well as validation data (for instance as a json file, as described below). You need to train your classifier with the training examples to find optimal parameters for the ``thetas`` used by ``parametrizedUnitary``. Let us denote the training examples as $\{(x_i, y_i)\}_i$, where each $x_i$ is a $2^n$-dimensional vector and each $y_i$ is either 0 or 1, labelling the two classes. Let 

$$\pi(x, \theta, b) = \hat{p}(0 | x, \theta) + b$$

where $\hat{p}(0 | x, \theta)$ is an estimate for the probability that the first qubit of $U(\theta) |\phi(x) \rangle$ is $0$, when measured in the computational basis. You can obtain this estimate by running the circuit from above ($U(\theta) S_x |0^n \rangle$) multiple times and seeing how many times you obtain $0$ when you measure the first qubit (note that here $\theta = (\theta_1, \theta_2, ... \theta_m)$ denotes the vector ``thetas`` from ``parametrizedUnitary``, where $m$ is the number of parametrized gates in that unitary). Additionally, $b$ is a bias term (that will be learned in addition to $\theta$), $-0.5 \leq b \leq 0.5$.

We can define the least-squares objective function:

$$ f(\theta, b) = \sum_{i} | \pi(x_i, \theta, b) - y_i |^2 $$

Find the values $\theta$ and $b$ for which $f$ is minimized. One way to do this is to use the [scipy.optimize](https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html) library in Python, though you're free to use whatever you like.

You can test your implementation with the [halfmoon dataset](https://github.com/microsoft/Quantum/blob/master/samples/machine-learning/half-moons/data.json). See how your results compare to those from the [Microsoft implementation](https://github.com/microsoft/Quantum/tree/master/samples/machine-learning/half-moons).